modified from https://huggingface.co/agents-course/notebooks/blob/main/unit2/llama-index/components.ipynb

In [ ]:
import os
from pathlib import Path

import chromadb
import llama_index
import nest_asyncio
from datasets import load_dataset
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.evaluation import FaithfulnessEvaluator
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.llms.gemini import Gemini
from llama_index.vector_stores.chroma import ChromaVectorStore


# Create a QueryEngine for retrieval augmented generation
Setting up the persona database

In [ ]:
dataset = load_dataset(path="dvilasuero/finepersonas-v0.1-tiny", split="train")

Path("data").mkdir(parents=True, exist_ok=True)
for i, persona in enumerate(dataset):
    with open(Path("data") / f"persona_{i}.txt", "w") as f:
        f.write(persona["persona"])

# Loading and embedding persona documents

In [ ]:
reader = SimpleDirectoryReader(input_dir="data")
documents = reader.load_data()
len(documents)

5000

In [ ]:
# create the pipeline with transformations
pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        GeminiEmbedding(model_name="models/text-embedding-004"),
    ]
)

# run the pipeline sync or async
nodes = await pipeline.arun(documents=documents[:10])
nodes

/tmp/ipykernel_19862/2472568562.py:5: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  GeminiEmbedding(model_name="models/text-embedding-004"),


[TextNode(id_='1fdc7978-0714-47d9-bed0-bf77361dfc43', embedding=[0.01253699, -0.0027807902, -0.019387979, 0.008286314, 0.0076110275, -0.0022635548, -0.00967554, 0.045222618, 0.022087712, 0.010916324, 0.022806427, 0.03225796, 0.047898576, -0.004488884, 0.018433727, -0.030189885, 0.01193961, 0.07902616, -0.06797857, 0.030058986, 0.021278389, -0.023753, 0.06576986, -0.077866524, -0.02478912, 0.0040420964, 0.017632432, -0.065125555, 0.013133193, -0.028327202, 0.06660359, 0.07079467, 0.029474389, -0.041483235, -0.048342705, 0.036194384, -0.014553471, -0.030912576, 0.03475663, -0.08971048, -0.03736995, 0.034817297, -0.0006029275, 0.056423984, -0.028031195, -0.051587917, 0.017305411, 0.020057151, -0.034446664, 0.042338382, 0.04965763, 0.010691435, -0.073634684, 0.046544958, -0.017319763, -0.016259253, -0.01704548, -0.029181512, 0.035139263, 0.008486483, 0.0425937, -0.009650146, -0.04083474, -0.059213202, -0.009019093, -0.047042936, 0.012571297, -0.011729005, -0.06593142, 0.045403365, -0.00893

# Storing and indexing documents

In [ ]:
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection(name="alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        GeminiEmbedding(model_name="models/text-embedding-004"),
    ],
    vector_store=vector_store,
)

nodes = await pipeline.arun(documents=documents[:10])
len(nodes)

/tmp/ipykernel_19862/4179033540.py:8: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  GeminiEmbedding(model_name="models/text-embedding-004"),


10

In [16]:
embed_model = GeminiEmbedding(model_name="models/text-embedding-004")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)

/tmp/ipykernel_19862/1365642989.py:1: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  embed_model = GeminiEmbedding(model_name="models/text-embedding-004")


# Querying the index

In [18]:
nest_asyncio.apply()  # This is needed to run the query engine
llm = Gemini(
    model="models/gemini-2.0-flash",
    # api_key="some key",  # uses GOOGLE_API_KEY env var by default
)
query_engine = index.as_query_engine(
    llm=llm,
    response_mode="tree_summarize",
)
response = query_engine.query(
    "Respond using a persona that describes author and travel experiences?"
)

response

/tmp/ipykernel_19862/2882912582.py:2: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(


Response(response='I am an environmental historian and urban planner dedicated to ecological conservation and sustainability. I probably work in local government or a similar institution.\n', source_nodes=[NodeWithScore(node=TextNode(id_='be93b13c-5d17-48d5-81e2-cb1813088cbe', embedding=None, metadata={'file_path': '/home/zihaog/repos/Coral_de_LLM_recipes/Hugging_Face_Agents_Course/unit2/llama-index/data/persona_1004.txt', 'file_name': 'persona_1004.txt', 'file_type': 'text/plain', 'file_size': 160, 'creation_date': '2025-03-25', 'last_modified_date': '2025-03-25'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3e9f40fe-9850-493d-8088-84f65cfc70ff', node_type='4', metadata={'file_path': '/home/zihaog/repo

# Evaluation and observability

In [19]:
# query index
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result.passing

True

In [ ]:
PHOENIX_API_KEY = "<PHOENIX_API_KEY>"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)

In [ ]:
response = query_engine.query(
    "What is the name of the someone that is interested in AI and techhnology?"
)
response